# Imports

In [1]:
import os
import json
import time
from tqdm import tqdm
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import pickle

import mlflow
import mlflow.sklearn
import optuna
from optuna.integration.mlflow import MLflowCallback

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import root_mean_squared_error

import shap

# Establish Development Iteration

In [2]:
ITER = 1

# Load train DataFrame

## Load DF and JSON

In [3]:
DF_TRAIN_DIR = os.path.join(
    os.getcwd(),
    '..',
    'data',
    'processed',
    'iter' + str(ITER),
    'train'
)
df_train_file = list(filter(lambda x: 'parquet' in x and 'dvc' not in x, os.listdir(DF_TRAIN_DIR)))[0]
train_json_file = list(filter(lambda x: 'json' in x and 'dvc' not in x, os.listdir(DF_TRAIN_DIR)))[0]

In [4]:
with open(os.path.join(DF_TRAIN_DIR, train_json_file), 'r') as f:
    train_json = json.load(f)

print(train_json['reporting_month'])

['2023/10/01', '2023/11/01', '2023/12/01', '2024/01/01', '2024/02/01', '2024/03/01', '2024/04/01', '2024/05/01', '2024/06/01', '2024/07/01', '2024/08/01', '2024/09/01', '2024/10/01']


In [5]:
# Read with PyArrow
table = pq.read_table(os.path.join(DF_TRAIN_DIR, df_train_file))

# Convert to pandas DataFrame
df_train = table.to_pandas()

In [6]:
df_train.head()

,reporting_month,ISRC,continent,zone,quantity,revenue
0,2023/10/01,GB-LFP-20-86366,Europe,Belarus,13,7.647710e-07
1,2023/10/01,FR-2X4-23-79996,Africa,Namibia,3,7.649240e-07
2,2023/10/01,FR-X20-23-35046,Asia,Nepal,3,7.649240e-07
3,2023/10/01,CA-5KR-21-69877,Europe,Malta,3,7.649240e-07
4,2023/10/01,CA-5KR-21-25668,Africa,Tunisia,3,7.649240e-07


# Load test DataFrame

In [7]:
DF_TEST_DIR = os.path.join(
    os.getcwd(),
    '..',
    'data',
    'processed',
    'iter' + str(ITER),
    'test'
)
df_test_file = list(filter(lambda x: 'parquet' in x and 'dvc' not in x, os.listdir(DF_TEST_DIR)))[0]

In [8]:
# Read with PyArrow
table = pq.read_table(os.path.join(DF_TEST_DIR, df_test_file))

# Convert to pandas DataFrame
df_test = table.to_pandas()

In [9]:
df_test.head()

,reporting_month,ISRC,continent,zone,quantity,revenue
0,2024/10/01,CA-5KR-21-25671,LATAM,Brazil,2,0.000949
1,2024/10/01,DG-A07-23-86106,Asia,Indonesia,10,0.000949
2,2024/10/01,FR-10S-24-80273,Europe,Romania,2,0.000953
3,2024/10/01,FR-10S-24-26330,Europe,Romania,2,0.000953
4,2024/10/01,UK-XN2-19-24890,Europe,Hungary,1,0.000954


# Prepare DataFrames for training
First, we will just drop "reporting_month" column, as we don't want to use it that information in our model. After that, we will check if there are unseen column values on test set. If so, we will replace them by the "UNKNOWN" word.

### Drop reporting_month

In [10]:
df_train = df_train.drop(columns='reporting_month')
df_train.head()

,ISRC,continent,zone,quantity,revenue
0,GB-LFP-20-86366,Europe,Belarus,13,7.647710e-07
1,FR-2X4-23-79996,Africa,Namibia,3,7.649240e-07
2,FR-X20-23-35046,Asia,Nepal,3,7.649240e-07
3,CA-5KR-21-69877,Europe,Malta,3,7.649240e-07
4,CA-5KR-21-25668,Africa,Tunisia,3,7.649240e-07


In [11]:
df_test = df_test.drop(columns='reporting_month')
df_test.head()

,ISRC,continent,zone,quantity,revenue
0,CA-5KR-21-25671,LATAM,Brazil,2,0.000949
1,DG-A07-23-86106,Asia,Indonesia,10,0.000949
2,FR-10S-24-80273,Europe,Romania,2,0.000953
3,FR-10S-24-26330,Europe,Romania,2,0.000953
4,UK-XN2-19-24890,Europe,Hungary,1,0.000954


### Replace unseen values on Test set

In [12]:
seen_isrc = set(train_json['ISRC'])
seen_zone = set(train_json['zone'])

In [13]:
# Replace unseen values using pandas where (faster for large datasets)
df_test['ISRC'] = df_test['ISRC'].where(
    df_test['ISRC'].isin(seen_isrc), 
    'UNKNOWN'
)

df_test['zone'] = df_test['zone'].where(
    df_test['zone'].isin(seen_zone), 
    'UNKNOWN'
)

In [14]:
# Check how many values were replaced
print(f"Unseen ISRCs replaced: {(df_test['ISRC'] == 'UNKNOWN').sum()}")
print(f"Unseen zones replaced: {(df_test['zone'] == 'UNKNOWN').sum()}")

Unseen ISRCs replaced: 1204
Unseen zones replaced: 2


# Train Models

## MLFlow setup

In [15]:
# Set MLflow tracking URI (local for notebook)
mlflow.set_tracking_uri("mlruns")

# Set experiment name
EXPERIMENT_NAME = "revenue_prediction_toy_training"
mlflow.set_experiment(EXPERIMENT_NAME)

print(f"MLflow tracking URI: {mlflow.get_tracking_uri()}")
print(f"Experiment: {EXPERIMENT_NAME}")

MLflow tracking URI: mlruns
Experiment: revenue_prediction_toy_training


/home/alumno/Desktop/datos/MLOps/MLOps_Final_Project/.venv/lib/python3.13/site-packages/mlflow/tracking/_tracking_service/utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)


## Data splits setup

In [16]:
TARGET_COLUMN = 'revenue'

X_train = df_train.drop(columns=[TARGET_COLUMN])
y_train = df_train[TARGET_COLUMN]

X_test = df_test.drop(columns=[TARGET_COLUMN])
y_test = df_test[TARGET_COLUMN]

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Feature columns: {list(X_train.columns)}")

X_train shape: (133211, 4)
X_test shape: (33302, 4)
Feature columns: ['ISRC', 'continent', 'zone', 'quantity']


### Encode Categorical Features

In [17]:
print(f"Available categorical columns: {list(train_json.keys())}\n")

# Apply transformations based on JSON category lists
for col in X_train.columns:
    if col in train_json:
        categories = train_json[col]
        
        # Add 'UNKNOWN' as index 0 if not already present
        if 'UNKNOWN' not in categories:
            categories = ['UNKNOWN'] + categories
        
        print(f"Encoding '{col}': {len(categories)} categories (including UNKNOWN)")
        
        # Create mapping from category to index
        category_to_idx = {cat: idx for idx, cat in enumerate(categories)}
        
        # Apply mapping to train and test
        X_train[col] = X_train[col].map(category_to_idx)
        X_test[col] = X_test[col].map(category_to_idx)
        
        # Check if there are still any NaN values (shouldn't happen now)
        if X_train[col].isna().any() or X_test[col].isna().any():
            n_unknown_train = X_train[col].isna().sum()
            n_unknown_test = X_test[col].isna().sum()
            print(f"  ERROR: Still have NaN values in '{col}' - Train: {n_unknown_train}, Test: {n_unknown_test}")
            # Fallback to -1 for any truly unknown values
            X_train[col] = X_train[col].fillna(-1)
            X_test[col] = X_test[col].fillna(-1)
    else:
        print(f"'{col}' not in JSON (keeping as numeric)")

print(f"\nEncoded X_train shape: {X_train.shape}")
print(f"Encoded X_test shape: {X_test.shape}")
print("\nFirst few rows after encoding:")
print(X_train.head())
print("\nData types after encoding:")
print(X_train.dtypes)

Available categorical columns: ['reporting_month', 'ISRC', 'zone', 'continent']

Encoding 'ISRC': 290 categories (including UNKNOWN)
Encoding 'continent': 8 categories (including UNKNOWN)
Encoding 'zone': 153 categories (including UNKNOWN)
'quantity' not in JSON (keeping as numeric)

Encoded X_train shape: (133211, 4)
Encoded X_test shape: (33302, 4)

First few rows after encoding:
   ISRC  continent  zone  quantity
0   257          3    14        13
1   173          1    95         3
2   203          2    96         3
3    58          3    85         3
4    29          1   139         3

Data types after encoding:
ISRC         int64
continent    int64
zone         int64
quantity     int64
dtype: object


In [18]:
# Calculate number of CV folds so validation split equals test size
# df_train is 80% of total data, df_test is 20% of total data
# We want validation to also be 20% of total data

total_data_size = len(df_train) + len(df_test)
test_size = len(df_test)
train_size = len(df_train)

# Validation should be 20% of total = test_size
# If we use k-fold CV, each fold is (1/k) of df_train
# We want (1/k) * train_size = test_size
# So k = train_size / test_size

k_folds = int(train_size / test_size)

print(f"Total data size: {total_data_size}")
print(f"Train size: {train_size} ({train_size/total_data_size*100:.1f}%)")
print(f"Test size: {test_size} ({test_size/total_data_size*100:.1f}%)")
print(f"Using {k_folds}-fold CV")
print(f"Each validation fold: ~{train_size/k_folds:.0f} samples ({(train_size/k_folds)/total_data_size*100:.1f}%)")

Total data size: 166513
Train size: 133211 (80.0%)
Test size: 33302 (20.0%)
Using 4-fold CV
Each validation fold: ~33303 samples (20.0%)


## Optuna objective function

In [19]:
def objective(trial):
    """
    Optuna objective function for hyperparameter optimization.
    Trains Random Forest with cross-validation and logs to MLflow.
    """
    
    # Suggest hyperparameters
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'random_state': 42
    }
    
    # Start MLflow run (nested under parent run)
    with mlflow.start_run(nested=True, run_name=f"trial_{trial.number}"):
        
        # Log hyperparameters
        mlflow.log_params(params)
        mlflow.log_param("cv_folds", k_folds)
        
        # Create model
        model = RandomForestRegressor(**params, n_jobs=2)
        
        # Train with cross-validation and track training time
        start_time = time.time()
        
        cv_results = cross_validate(
            model,
            X_train,
            y_train,
            cv=k_folds,
            scoring='neg_root_mean_squared_error',
            return_train_score=True,
            n_jobs=2
        )
        
        training_time = time.time() - start_time
        
        # Calculate average metrics
        train_rmse = -cv_results['train_score'].mean()
        val_rmse = -cv_results['test_score'].mean()
        
        # Log CV metrics
        mlflow.log_metric("train_rmse_cv", train_rmse)
        mlflow.log_metric("val_rmse_cv", val_rmse)
        mlflow.log_metric("training_time_seconds", training_time)
        
        # Log per-fold metrics
        for fold_idx in range(k_folds):
            mlflow.log_metric(f"fold_{fold_idx}_train_rmse", 
                            -cv_results['train_score'][fold_idx])
            mlflow.log_metric(f"fold_{fold_idx}_val_rmse", 
                            -cv_results['test_score'][fold_idx])
        
        # Train final model on full training set
        model.fit(X_train, y_train)
        
        # Evaluate on test set
        eval_start_time = time.time()
        y_pred_test = model.predict(X_test)
        eval_time = time.time() - eval_start_time
        
        test_rmse = root_mean_squared_error(y_test, y_pred_test)
        
        # Log test metrics
        mlflow.log_metric("test_rmse", test_rmse)
        mlflow.log_metric("evaluation_time_seconds", eval_time)
        
        # Log model with MLflow
        mlflow.sklearn.log_model(model, "random_forest_model")
        
        # Save and log model as pickle
        model_path = f"models/trial_{trial.number}_model.pkl"
        os.makedirs("models", exist_ok=True)
        with open(model_path, 'wb') as f:
            pickle.dump(model, f)
        mlflow.log_artifact(model_path, "model_artifacts")
        
        # Feature importance plot
        feature_importance = pd.DataFrame({
            'feature': X_train.columns,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        plt.figure(figsize=(10, 6))
        sns.barplot(data=feature_importance.head(10), x='importance', y='feature')
        plt.title(f'Top 10 Feature Importances - Trial {trial.number}')
        plt.tight_layout()
        importance_plot_path = f"plots/trial_{trial.number}_feature_importance.png"
        os.makedirs("plots", exist_ok=True)
        plt.savefig(importance_plot_path, dpi=300, bbox_inches='tight')
        plt.close()
        mlflow.log_artifact(importance_plot_path, "feature_importance")
        
        # SHAP values (on a sample for speed in toy training)
        sample_size = min(100, len(X_train))
        X_sample = X_train.sample(n=sample_size, random_state=42)
        
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_sample)
        
        # SHAP summary plot
        plt.figure(figsize=(10, 8))
        shap.summary_plot(shap_values, X_sample, show=False)
        shap_plot_path = f"plots/trial_{trial.number}_shap_summary.png"
        plt.savefig(shap_plot_path, dpi=300, bbox_inches='tight')
        plt.close()
        mlflow.log_artifact(shap_plot_path, "shap_analysis")
        
        # SHAP feature importance
        plt.figure(figsize=(10, 6))
        shap.summary_plot(shap_values, X_sample, plot_type="bar", show=False)
        shap_importance_path = f"plots/trial_{trial.number}_shap_importance.png"
        plt.savefig(shap_importance_path, dpi=300, bbox_inches='tight')
        plt.close()
        mlflow.log_artifact(shap_importance_path, "shap_analysis")
        
        # Log dataset metadata files (if they exist)
        dataset_dvc_path = os.path.join(DF_TRAIN_DIR, df_train_file + '.dvc')
        dataset_json_path = os.path.join(DF_TRAIN_DIR, train_json_file)
        
        if os.path.exists(dataset_dvc_path):
            mlflow.log_artifact(dataset_dvc_path, "dataset_metadata")
        
        if os.path.exists(dataset_json_path):
            mlflow.log_artifact(dataset_json_path, "dataset_metadata")
        
        # Create and log hyperparameters JSON
        hyperparams_dict = {
            **params,
            'cv_folds': k_folds,
            'trial_number': trial.number
        }
        hyperparams_path = f"configs/trial_{trial.number}_hyperparams.json"
        os.makedirs("configs", exist_ok=True)
        with open(hyperparams_path, 'w') as f:
            json.dump(hyperparams_dict, f, indent=4)
        mlflow.log_artifact(hyperparams_path, "hyperparameters")
        
        print(f"Trial {trial.number}: val_rmse={val_rmse:.4f}, test_rmse={test_rmse:.4f}")
        
    # Return validation RMSE for Optuna to optimize
    return val_rmse

## Launch Optuna Study with MLFlow

In [24]:
# Safety: end any active runs before starting
mlflow.end_run()

# Create parent MLflow run
with mlflow.start_run(run_name="optuna_optimization"):
    
    # Log study configuration
    mlflow.log_param("optimization_metric", "val_rmse")
    mlflow.log_param("n_trials", 5)
    mlflow.log_param("model_type", "RandomForest")
    
    # Create Optuna study
    study = optuna.create_study(
        study_name="revenue_rf_optimization",
        direction="minimize",
        sampler=optuna.samplers.TPESampler(seed=42)
    )
        
    # Run optimization
    print("Starting Optuna optimization...")
    study.optimize(
        objective,
        n_trials=5,
        show_progress_bar=True
    )
    
    # Log best trial information
    best_trial = study.best_trial
    mlflow.log_params({f"best_{k}": v for k, v in best_trial.params.items()})
    mlflow.log_metric("best_val_rmse", best_trial.value)
    
    print(f"\n{'='*60}")
    print("Optimization Complete!")
    print(f"{'='*60}")
    print(f"Best trial number: {best_trial.number}")
    print(f"Best validation RMSE: {best_trial.value:.4f}")
    print(f"Best hyperparameters:")
    for key, value in best_trial.params.items():
        print(f"  {key}: {value}")

[I 2026-01-01 17:29:01,997] A new study created in memory with name: revenue_rf_optimization


Starting Optuna optimization...


  0%|          | 0/5 [00:00<?, ?it/s]

2026/01/01 17:29:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/01 17:29:56 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Trial 0: val_rmse=0.6822, test_rmse=0.4416
[I 2026-01-01 17:30:20,078] Trial 0 finished with value: 0.6821804569154013 and parameters: {'n_estimators': 106, 'max_depth': 20}. Best is trial 0 with value: 0.6821804569154013.


...

Trial 2: val_rmse=0.7815, test_rmse=0.5944
[I 2026-01-01 17:31:34,886] Trial 2 finished with value: 0.7815487975104642 and parameters: {'n_estimators': 73, 'max_depth': 7}. Best is trial 0 with value: 0.6821804569154013.


2026/01/01 17:31:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/01 17:32:01 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Trial 3: val_rmse=0.6912, test_rmse=0.4391
[I 2026-01-01 17:32:08,411] Trial 3 finished with value: 0.6912160105087612 and parameters: {'n_estimators': 58, 'max_depth': 18}. Best is trial 0 with value: 0.6821804569154013.


2026/01/01 17:32:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/01 17:33:02 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Trial 4: val_rmse=0.6823, test_rmse=0.4447
[I 2026-01-01 17:33:10,794] Trial 4 finished with value: 0.6822787318872545 and parameters: {'n_estimators': 140, 'max_depth': 16}. Best is trial 0 with value: 0.6821804569154013.

Optimization Complete!
Best trial number: 0
Best validation RMSE: 0.6822
Best hyperparameters:
  n_estimators: 106
  max_depth: 20


## Visualize Optimization Results

In [25]:
# Plot optimization history
fig1 = optuna.visualization.plot_optimization_history(study)
fig1.write_image("plots/optimization_history.png")
mlflow.log_artifact("plots/optimization_history.png", "optimization_plots")
fig1.show()

# Plot parameter importances
fig2 = optuna.visualization.plot_param_importances(study)
fig2.write_image("plots/param_importances.png")
mlflow.log_artifact("plots/param_importances.png", "optimization_plots")
fig2.show()

# Plot parallel coordinate
fig3 = optuna.visualization.plot_parallel_coordinate(study)
fig3.write_image("plots/parallel_coordinate.png")
mlflow.log_artifact("plots/parallel_coordinate.png", "optimization_plots")
fig3.show()